<a href="https://colab.research.google.com/github/anubhavshrestha/Machine-Learning/blob/main/Task3_ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# import necessary modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [5]:
# defining necessary transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
# Load the CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)

# Split the training dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [55]:
# Defining Bottlenet architecture similar as Residual Block
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [56]:
# General architecture of ResNet
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [58]:
# Initialize the ResNet-50 model
model = ResNet(Bottleneck, [3, 4, 6, 3])  #specific implementation for ResNet 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and  optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [59]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = 100 * correct / total

    print(f"Epoch {epoch + 1} - Training Loss: {running_loss / len(train_loader):.3f}, Validation Accuracy: {val_accuracy:.2f}%")

print("Finished Training")

Epoch 1 - Training Loss: 1.908, Validation Accuracy: 42.59%
Epoch 2 - Training Loss: 1.347, Validation Accuracy: 52.61%
Epoch 3 - Training Loss: 1.079, Validation Accuracy: 63.34%
Epoch 4 - Training Loss: 0.886, Validation Accuracy: 69.84%
Epoch 5 - Training Loss: 0.750, Validation Accuracy: 73.36%
Epoch 6 - Training Loss: 0.655, Validation Accuracy: 74.20%
Epoch 7 - Training Loss: 0.581, Validation Accuracy: 76.53%
Epoch 8 - Training Loss: 0.532, Validation Accuracy: 76.18%
Epoch 9 - Training Loss: 0.490, Validation Accuracy: 82.24%
Epoch 10 - Training Loss: 0.447, Validation Accuracy: 80.10%
Finished Training


In [60]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Accuracy on the test set: 80.11%


#TRANSFER LEARNING USING PRE-TRAINED RESNET-50

In [63]:
#Transfer Learning on ResNet-50
from torchvision.models import resnet50

# Replace the last fully connected layer with a new one for CIFAR-10 (10 classes)
model = resnet50(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [64]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = 100 * correct / total

    print(f"Epoch {epoch + 1} - Training Loss: {running_loss / len(train_loader):.3f}, Validation Accuracy: {val_accuracy:.2f}%")

print("Finished Training")

Epoch 1 - Training Loss: 1.167, Validation Accuracy: 71.99%
Epoch 2 - Training Loss: 0.734, Validation Accuracy: 75.92%
Epoch 3 - Training Loss: 0.610, Validation Accuracy: 79.24%
Epoch 4 - Training Loss: 0.539, Validation Accuracy: 80.69%
Epoch 5 - Training Loss: 0.487, Validation Accuracy: 81.62%
Epoch 6 - Training Loss: 0.443, Validation Accuracy: 82.30%
Epoch 7 - Training Loss: 0.403, Validation Accuracy: 82.84%
Epoch 8 - Training Loss: 0.376, Validation Accuracy: 83.76%
Epoch 9 - Training Loss: 0.348, Validation Accuracy: 83.56%
Epoch 10 - Training Loss: 0.316, Validation Accuracy: 83.60%
Epoch 11 - Training Loss: 0.306, Validation Accuracy: 84.13%
Epoch 12 - Training Loss: 0.291, Validation Accuracy: 84.15%
Epoch 13 - Training Loss: 0.269, Validation Accuracy: 83.94%
Epoch 14 - Training Loss: 0.259, Validation Accuracy: 84.67%
Epoch 15 - Training Loss: 0.238, Validation Accuracy: 84.67%
Epoch 16 - Training Loss: 0.227, Validation Accuracy: 85.02%
Epoch 17 - Training Loss: 0.215, 